In [1]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import bokeh
from bokeh.models import ColumnDataSource, HoverTool, CrosshairTool, ranges, BoxAnnotation, Label, Div
from bokeh.plotting import figure, show, column, gridplot
from datetime import datetime
from bokeh.themes import built_in_themes
from bokeh.io import output_notebook, output_file
output_notebook()

Loading BokehJS ...

In [2]:
tickers = ['BP','ENB','SHEL', 'PBR']

names = {'BP': 'British Petroleum',
         'ENB':'Enbridge Inc',
         'SHEL':'Shell',
         'PBR':'Petrobras'}

colors = {'BP': '#0072B2',
         'ENB':'#D55E00',
         'SHEL':'#009E73',
         'PBR':'#CC79A7'}

### Oil Companies Functions

In [3]:
start = datetime(2018, 1, 1)
end = datetime(2022, 10, 29)
closing_price = web.DataReader(tickers, 'yahoo', start, end)
closing_price.head()

Attributes  Adj Close                                      Close             \
Symbols            BP        ENB       SHEL       PBR         BP        ENB   
Date                                                                          
2018-01-02  32.144497  30.779902  54.317707  5.873280  42.380001  40.139999   
2018-01-03  32.516151  30.986935  54.756783  6.021485  42.869999  40.410000   
2018-01-04  32.645096  31.262991  54.996292  6.037951  43.040001  40.770000   
2018-01-05  32.690605  31.239986  55.116032  6.081863  43.099998  40.740002   
2018-01-08  32.675438  31.232323  55.132000  6.158710  43.080002  40.730000   

Attributes                         High             ...        Low         \
Symbols          SHEL    PBR         BP        ENB  ...       SHEL    PBR   
Date                                                ...                     
2018-01-02  68.040001  10.70  42.430000  40.220001  ...  66.910004  10.43   
2018-01-03  68.589996  10.97  42.889999  40.500000  ...  67.919998  10.63   
2018-01-04  68.889999  11.00  43.169998  40.799999  ...  68.599998  10.94   
2018-01-05  69.040001  11.08  43.150002  41.209999  ...  68.577499  10.89   
2018-01-08  69.059998  11.22  43.119999  40.860001  ...  68.690002  11.02   

Attributes       Open                                  Volume             \
Symbols            BP        ENB       SHEL    PBR         BP        ENB   
Date                                                                       
2018-01-02  42.060001  39.360001  67.000000  10.46  4204900.0  2464100.0   
2018-01-03  42.430000  40.290001  67.919998  10.80  4753600.0  1839000.0   
2018-01-04  43.009998  40.590000  68.620003  11.00  6113800.0  2025700.0   
2018-01-05  43.049999  41.040001  68.790001  10.99  3846400.0  2499700.0   
2018-01-08  42.990002  40.750000  68.800003  11.06  3156200.0  1497000.0   

Attributes                         
Symbols          SHEL         PBR  
Date                               
2018-01-02  3470654.0  14023400.0  
2018-01-03  3483320.0  21504100.0  
2018-01-04  2638891.0  22945100.0  
2018-01-05  2562434.0  12230100.0  
2018-01-08  2741832.0  13495800.0  

[5 rows x 24 columns]

### Helper Functions & Data Source

In [4]:
def create_figure() -> bokeh.plotting.Figure:
    p = figure(x_axis_type = "datetime", width  = 800, height = 300,
          tools="pan, box_select,wheel_zoom,reset,save",
          x_axis_label = 'Date', y_axis_label = 'Price per Share ($USD)')
    return p
    
def format_plot(p: bokeh.plotting.Figure) -> None:
    
    p.border_fill_color = None
    p.outline_line_color = 'black'
    p.legend.border_line_color = 'black'
    p.add_layout(p.legend[0], 'right')
    p.y_range = ranges.Range1d(0, 75)
    p.yaxis.minor_tick_line_color = None
    p.toolbar_location = None

    
def add_tools_to_plot(p: bokeh.plotting.Figure) -> None:
    """Adds HoverTool and CrosshairTool to plot."""
    h = HoverTool(
            tooltips=[("Stock", "$name"),
                      ("Date", "$x{%F}"),
                      ("Close Price", "$y"),
                     ],
            formatters = { "$x": 'datetime'},
            show_arrow = True
    )
    c = CrosshairTool(dimensions = 'width')
    return

def add_band(p: bokeh.plotting.Figure) -> None:
    """Adds a BoxAnnotation and text label for 2020 stock market crash. """
    
    box = BoxAnnotation(left = datetime(2020, 2, 17 ), top = 70, right = datetime(2020, 4, 7),        
            fill_alpha=0.4, line_width=1, line_color=None, fill_color = 'red')
    label = Label(
        x=datetime(2020, 4, 7 ),
        y=70, text='COVID-19 Crash',
        text_align = 'center', 
        text_color = 'black', 
        text_font = 'Arial', 
        text_font_size = '10px',
        text_font_style = 'italic'
                 )
    
    p.add_layout(box)
    p.add_layout(label)

datasource = ColumnDataSource(closing_price['Close'])

### Legend in Alphabetical Order

In [5]:
p1 = create_figure()
for tick, cl in zip(sorted(tickers), colors):
    p1.line('Date', tick, source = datasource, name = names[tick], 
           line_width = 2, line_color= colors[tick], legend_label = names[tick])

format_plot(p1)
add_tools_to_plot(p1)
add_band(p1)
show(p1)

### Legend in Data Order

In [6]:
filt_2022 = closing_price.index > datetime(2022, 1, 1)
avg_prices_2022 = closing_price.loc[filt_2022, 'Close'].mean().sort_values(ascending = False)
avg_prices_2022

Symbols
SHEL    53.069423
ENB     42.660577
BP      30.373221
PBR     13.560433
dtype: float64

In [7]:
p2 = create_figure()
for tick, cl in zip(avg_prices_2022.index, colors):
    p2.line('Date', tick, source = datasource, name = names[tick], 
           line_width = 2, line_color= colors[tick], legend_label = names[tick])

format_plot(p2)
add_tools_to_plot(p2)
add_band(p2)
show(p2)

### Column Layout

In [14]:
p1.xaxis.axis_label = None
p1.xaxis.major_label_text_font_size = '0pt'
p1.title.text = ' Stock price over time for four major Oil companies'
layout = gridplot([[p1], [Div()], [p2]])
show(layout)